In [21]:
# importacoes

from requests import Session
import pandas as pd
import re
from html import unescape
from requests.exceptions import ChunkedEncodingError, ReadTimeout, RequestException
from time import sleep

In [22]:
# declaracoes

session = Session()
def get_content_president(year, month, day):
  for attempt in range(3):
    try:
      response = session.get(f"https://www.bcb.gov.br/api/servico/sitebcb/agendadiretoria?lista=Agenda%20da%20Diretoria&inicioAgenda=%272023-02-27%27&fimAgenda=%27{year}-{month}-{day}%27", timeout=10)
      if (response.status_code == 200):
        data = response.json()
        if (data["conteudo"][0]["idAutoridade"] == 39):
          return data["conteudo"][0]
    except (ChunkedEncodingError, ReadTimeout, RequestException):
      if attempt < 2:
        sleep(2)
      else:
        raise
  return None


descriptions = {}  # { "sheet_name": [...descriptions], ... }
dfs_sheets = {}  # { "sheet_name": df, ... }

In [24]:
# obtencao dos dados de 2023
year = 2023
for month in range(2, 4):
  sheet_name = f"{month}-{year}"
  descriptions[sheet_name] = []
  if month == 2:
    content = get_content_president(year, month, 28)
    descriptions[sheet_name].append(content["descricao"])
  elif month in [3, 5, 7, 8, 10, 12]:
    for day in range(1, 32):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])
  elif month in [4, 6, 9, 11]:
    for day in range(1, 31):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])

In [ ]:
# obtencao dos dados de 2024
year = 2024
for month in range(5, 7):
  sheet_name = f"{month}-{year}"
  descriptions[sheet_name] = []
  if month in [1, 3, 5]:
    for day in range(1, 32):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])
  elif month == 4:
    for day in range(1, 31):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])
  elif month == 2:
    for day in range(1, 30):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])
  elif month == 6:
    for day in range(1, 29):
      content = get_content_president(year, month, day)
      descriptions[sheet_name].append(content["descricao"])

In [ ]:
for sheet_name, descriptions_texts in descriptions.items:
  # limpeza dos dados
  descriptions_texts_normalized = [re.sub(r'<[^>]+>', '', description.encode("utf-8").decode("unicode_escape")) for description in descriptions_texts]

  # criacao das tabelas para exibicao dos dados
  df = pd.DataFrame({
    "descricao do evento": descriptions_texts_normalized
  })
  dfs_sheets[sheet_name] = df

In [ ]:
# exportacao dos dados
excel_file_name = "agenda.xlsx"

df_xlsx = pd.read_excel(excel_file_name)
df_xlsx_no_empty_cells = df_xlsx.dropna(how="any").dropna(axis=1, how="any")
df_xlsx_no_empty_cells.to_excel(excel_file_name, index=False)

with pd.ExcelWriter(excel_file_name) as writer:
  for sheet_name, df in dfs_sheets.items():
    df.to_excel(writer, sheet_name=sheet_name)